# This notebook is designed and tested using RunPod.com

*Pytorch 2.0.1 template:  
  runpod/pytorch:2.0.1-py3.10-cuda11.8.0-devel-ubuntu22.04*

In [ ]:
##############################################################################################################
# set environment variables
##############################################################################################################

import os

print("Setting environment variables...")
os.environ['GIT_REPO'] = 'https://github.com/TechTailor32/roop-unleashed.git'
os.environ['NAME'] = 'ROOP'
os.environ['WORKINGFOLDER'] = '/workspace/roop-unleashed'  # Change to a valid path (e.g., /kaggle/workspace/roop-unleashed for Kaggle)
os.environ['PLATFORM'] = 'runpod'   # other options is 'kaggle'
os.environ['NGROKTOKEN'] = ""  # if no token is here, system will run by default and use Gradio.  Otherwise, paste token between quotes and you can use NGROK as well
os.environ['NGROKDOMAIN'] = ""  # optional, leave empty if you don't have an ngrok domain
# CONFIGFILE: config_cpu.yaml (CPU only) config_gpu.yaml (GPU) config_colab.yaml (Google Colab or Kaggle)
os.environ['CONFIGFILE'] = 'config_cuda.yaml'  # this default is config_cuda.yaml assuming you have GPU

# Output Status to Screen
print("Environment variables set:")
print("GIT_REPO:", os.environ['GIT_REPO'])
print("NAME:", os.environ['NAME'])
print("WORKINGFOLDER:", os.environ['WORKINGFOLDER'])
print("CONFIGFILE:", os.environ['NGROKTOKEN'])
print("CONFIGFILE:", os.environ['NGROKDOMAIN'])
print("CONFIGFILE:", os.environ['CONFIGFILE'])


In [ ]:
##############################################################################################################
# intall git and clone the repo (set in the Environment Variables section)
#   and output status to screen
##############################################################################################################

import os

print("Installing GitPython library...")
os.system('apt-get install -y git')
os.system("pip install gitpython")
print("GitPython installed successfully.")

from git import Repo
from git.exc import GitCommandError

# Environment variables
GIT_REPO_URL = os.environ['GIT_REPO']
WORKING_FOLDER = os.environ['WORKINGFOLDER']

try:
    print("Cloning the repository...")
    Repo.clone_from(GIT_REPO_URL, WORKING_FOLDER)
    print(f"Repository cloned successfully into {WORKING_FOLDER}")
except GitCommandError as e:
    print(f"Failed to clone the repository. Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [ ]:
##############################################################################################################
# Show current folder -> Change to the working directory if not there
##############################################################################################################

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Check if the current directory is the same as the WORKINGFOLDER
if current_directory != os.environ['WORKINGFOLDER']:
    print("Changing directory to the WORKINGFOLDER...")
    os.chdir(os.environ['WORKINGFOLDER'])
    # Confirm the change of directory
    print("New Current Working Directory:", os.getcwd())
else:
    print("Already in the WORKINGFOLDER.")


In [ ]:
##############################################################################################################
# swap the config file based on choices in Environment Variables section
##############################################################################################################

import shutil

print("Moving configuration file...")
source_path = os.path.join(os.environ['WORKINGFOLDER'], os.environ['CONFIGFILE'])
destination_path = os.path.join(os.environ['WORKINGFOLDER'], 'config.yaml')
shutil.move(source_path, destination_path)
print(f"Moved {os.environ['CONFIGFILE']} to {destination_path}")


In [ ]:
##############################################################################################################
# install extra dependencies and libraries
##############################################################################################################

import os

print("Updating system and installing required packages...")
os.system("apt-get update")
os.system("apt-get install -y libgl1-mesa-glx ffmpeg")
print("Packages installed successfully.")

# Assume PLATFORM and CONFIGFILE are environment variables
PLATFORM = os.getenv('PLATFORM')
CONFIGFILE = os.getenv('CONFIGFILE')

if PLATFORM == 'kaggle':
    if CONFIGFILE == 'config_cuda.yaml':
        print("CUDA configuration detected. Installing CUDA drivers...")
        os.system("apt-get install --no-install-recommends cuda-drivers")
    else:
        print("No CUDA configuration detected. Proceeding without CUDA installation.")
else:
    print("Finished checking platform and configuration.")




In [ ]:
##############################################################################################################
# install dependencies based on the requirements.txt in the repo
##############################################################################################################

import os

print("Installing required Python packages from requirements.txt...")
# Construct the command string
command = "pip install --no-cache-dir -r " + os.path.join(os.environ["WORKINGFOLDER"], "requirements.txt")
# Execute the command
os.system(command)
print("Python packages installed.")



In [ ]:
##############################################################################################################
#  the repo has Gradio support.  The following checks for an ngrok as well.
##############################################################################################################

import os
import subprocess

# Retrieve environment variables
NGROK_TOKEN = os.getenv('NGROKTOKEN')
NGROK_DOMAIN = os.getenv('NGROKDOMAIN')

# Check if NGROK_TOKEN is set
if NGROK_TOKEN:
    # Install pyngrok if not already installed
    subprocess.call(["pip", "install", "pyngrok"])

    from pyngrok import ngrok, conf

    # Ensure no ngrok instances are running
    ngrok.kill()

    # Set up and open an ngrok tunnel
    ngrok_config = conf.PyngrokConfig(auth_token=NGROK_TOKEN, region="us")
    public_url = ngrok.connect(7860, pyngrok_config=ngrok_config, bind_tls=True, domain=NGROK_DOMAIN).public_url
    print("Ngrok Tunnel URL:", public_url)

    # Run the Python script
    os.system("python run.py")
else:
    print("No ngrok token detected, running default system...")
    os.system("python run.py")